In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# Expectations

In [ ]:
#Import POS file, AMT, Pallet Quantity file, Sku Conversion Sheet from THD to Orders

#Use AMT sheet, merge in pallet counts per sku. Assign this a new variable "OOS_Report".

#To "OOS Report" merge in POS OH units, fill na with 0. 

#Create a new column, labelled "OH PLTs". This column is the rounded of (OH Units / Pallet counts)

#Check for orders, import the Orders data. Merge the sku conversion to the file.
#Change the customer number to store number. 

#Combine the open with the closed list. 

#Merge the sum of the totals by store by sku with the "OOS_Report" file. 

#Create boolean filter to take away anything that total doesn't equal 0 pallets.

# Live Code

In [ ]:
POS_File = pd.read_excel()
AMT = pd.read_excel()
Sku_Info = pd.read_excel()

In [ ]:
POS_File.head(3)

In [ ]:
AMT.head(3)

In [ ]:
Sku_Info.head(3)

In [ ]:
#Code Block For All Out Of Stock Skus Report - No Open Order Information

AMT.drop(columns = ['STR NAME', 'SET NAME'], inplace = True)

AMT = AMT[AMT['AMT'] == 'Y']

OOS_Report = AMT.merge(Sku_Info, how = 'left', left_on = 'Sku', right_on = 'ITM')
OOS_Report.drop(columns = ['Item', 'Description', 'ITM', 'ITM DESC'], inplace = True)

OOS_Report['PLT'] = OOS_Report['PLT'].astype('int')

OOS_Report = OOS_Report.merge(POS_File, how = 'left', left_on = ['STR NBR', 'Sku'], right_on = ['Store', 'Item'])

OOS_Report.drop(columns = ['BYO', 'Market', 'Store', 'Item', 'Item Description'], inplace = True)

OOS_Report = OOS_Report.fillna(0)

OOS_Report[2019] = OOS_Report[2019].astype('int')

OOS_Report.columns = ['BYO NBR', 'MKT NBR', 'STR NBR', 'Sku', 'Sku Description', 'AMT', 'PLT', 'OH Units']

OOS_Report['OH PLTs'] = (OOS_Report['OH Units'] / OOS_Report['PLT']).round().astype('int')

True_OOS = OOS_Report[OOS_Report['OH PLTs'] == 0]

In [ ]:
#Export Report --- This file is all out of stocks regardless of open orders
True_OOS.to_excel('All OOS skus.xlsx', index = False)

# Continue: Adding In On-Orders

In [ ]:
#Import Orders Open & Closed Orders
Orders = pd.read_excel()

In [ ]:
#Code Block To Follow Up True_OOS Report But Only Show OOS Skus Without An Open Order

Orders = Orders['Open'].append(Orders['Closed'], ignore_index=True, sort='True')

Orders.drop(columns = ['Segment1', 'Segment1 Description', 'Segment2', 'Segment2 Description', 'Segment3', 'Segment3 Description'], inplace = True)

Orders-Totals = Orders.groupby(by = ['Cust', 'Item']).sum()

Orders-Totals = Orders-Totals.drop_duplicates(keep = 'first')

Orders-Totals.drop(columns = ['Count Per Pallet', 'Customer PO Number', 'Ext Amt', 'SO Nbr - Rel', 'Unit Price'], inplace = True)

Orders-Totals.reset_index(inplace = True)

Orders-Totals['Cust'] = [x - 4560000 for x in Orders-Totals['Cust']]

Orders-Totals = Orders-Totals.merge(Sku_Info, how = 'left', on = 'Item')
Orders-Totals.drop(columns = ['Description', 'ITM DESC', 'PLT'], inplace = True)

Orders-Totals.dropna(how = 'any', inplace = True)

Orders-Totals['ITM'] = Orders-Totals['ITM'].astype('int')

POS_With_OO = OOS_Report.merge(Orders-Totals, how = 'left', left_on = ['STR NBR', 'Sku'], right_on = ['Cust', 'ITM']).fillna(0)

POS_With_OO.drop(columns = ['Cust', 'Item', 'ITM'], inplace = True)

POS_With_OO['OH & OO PLT'] = POS_With_OO['OH PLTs'] + POS_With_OO['Pallet Ordered']

Actual_OO = POS_With_OO[POS_With_OO['OH & OO PLT'] == 0]

In [ ]:
#Export Report --- This file is all out of stocks after open orders
Actual_OO.to_excel('OOS without Orders.xlsx', index = False)

# Plotting The Results

In [ ]:
plt.figure(1,figsize=(18,6))

plt.subplot(1,2,1)
True_OOS['Sku'].value_counts().plot(kind = 'bar', title = 'All OOS Skus', fontsize = 15)

plt.subplot(1,2,2)
Actual_OO['Sku'].value_counts().plot(kind = 'bar', title = 'OOS Skus Without Orders',fontsize = 15)